<a href="https://colab.research.google.com/github/LilliLee-1318/UIUC-helper-chatbot/blob/main/Only_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ======================================================
# 🎓 UIUC Assistant (Grounded RAG Version)
# ======================================================

!pip install -q transformers sentence-transformers faiss-cpu gradio bitsandbytes accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss, numpy as np
import requests
import gradio as gr

# ------------------------------------------------------
# 1️⃣ 모델 불러오기 (A100 GPU + 4bit 양자화)
# ------------------------------------------------------
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16
)

print("✅ Model loaded successfully on A100!")

# ------------------------------------------------------
# 2️⃣ 문장 임베딩 모델 로드
# ------------------------------------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ------------------------------------------------------
# 3️⃣ 카테고리 감지기 (질문 분류)
# ------------------------------------------------------
def detect_category(user_input):
    text = user_input.lower()
    if any(word in text for word in ["visa", "opt", "cpt", "f-1", "h-1", "isss", "tuition", "fee"]):
        return "admin"
    elif any(word in text for word in ["class", "course", "professor", "rate", "grade"]):
        return "course"
    elif any(word in text for word in ["bus", "dining", "menu", "meal", "dorm", "housing"]):
        return "lifestyle"
    else:
        return "general"

# ------------------------------------------------------
# 4️⃣ 데이터 수집 함수 (현재는 더미 데이터)
# ------------------------------------------------------

# ⚙️ 4-1. 행정지원 (ISSS, OPT, Tuition)
def fetch_admin_data():
    return [
        "OPT (Optional Practical Training) allows F-1 students to work in the U.S. for up to 12 months after completing their degree.",
        "To apply for OPT, students must first request an I-20 from the ISSS office.",
        "CPT (Curricular Practical Training) is for work experience that is part of your academic program.",
        "The tuition for undergraduate international students at UIUC is approximately $37,000 per year."
    ]

# ⚙️ 4-2. 수업 선택 (Course, Grade, Professor)
def fetch_course_data():
    return [
        "RateMyProfessor reviews can help identify professors with high teaching ratings.",
        "Grade disparity data for UIUC courses is available through the Course Explorer.",
        "CS 124 uses Kotlin as its main programming language.",
        "PSYC 210 is taught by professors with a 4.7 average rating on RateMyProfessor."
    ]

# ⚙️ 4-3. 생활 팁 (Housing, Dining, Bus)
def fetch_lifestyle_data():
    return [
        "The Ikenberry Dining Hall serves breakfast from 7 AM to 10 AM.",
        "ISR Dining offers vegetarian and vegan-friendly options.",
        "MTD buses are free for students and run every 10-15 minutes during weekdays.",
        "Dorms like Nugent and Hopkins are part of the Ikenberry Commons on the west side of campus."
    ]

# ------------------------------------------------------
# 5️⃣ RAG 검색 및 답변 생성
# ------------------------------------------------------
def retrieve_and_answer(user_input, docs):
    # 1️⃣ 문서 임베딩
    doc_embeds = embedder.encode(docs)
    index = faiss.IndexFlatL2(doc_embeds.shape[1])
    index.add(np.array(doc_embeds))

    # 2️⃣ 유사 문장 검색
    query_vec = embedder.encode([user_input])
    _, idxs = index.search(np.array(query_vec), k=3)
    retrieved_docs = [docs[i] for i in idxs[0]]
    context = "\n".join(retrieved_docs)

    # 3️⃣ 모델 입력 (근거 기반 프롬프트)
    prompt = f"""
You are a helpful UIUC assistant.
Use only the information from the context below.
Do not invent information or guess.
Cite specific details if possible.

Context:
{context}

Question:
{user_input}

Answer:
"""

    # 4️⃣ 모델 실행
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return answer

# ------------------------------------------------------
# 6️⃣ 메인 챗봇 함수
# ------------------------------------------------------
def chat_with_bot(user_input):
    category = detect_category(user_input)

    if category in ["admin", "course", "lifestyle"]:
        if category == "admin":
            docs = fetch_admin_data()
        elif category == "course":
            docs = fetch_course_data()
        else:
            docs = fetch_lifestyle_data()
        return retrieve_and_answer(user_input, docs)

    # 💬 일반 대화 모드
    prompt = f"""
You are a warm, conversational chatbot for UIUC students.
Your goal is to make students feel comfortable when chatting with you.
If the user greets you (e.g., "hello", "hi", "hey"), respond with a friendly, short greeting.
Do NOT give university information unless directly asked.
Keep your response casual and human-like.

User: {user_input}
Assistant:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Assistant:" in answer:
        answer = answer.split("Assistant:")[-1].strip()

    return answer

# ------------------------------------------------------
# 7️⃣ Gradio UI 설정 (밝은 UIUC 블루 적용)
# ------------------------------------------------------
uiuc_blue = "#1E3A8A"
uiuc_orange = "#FF552E"

with gr.Blocks(css=f"""
body {{
    background-color: #f8f9fb;
    font-family: 'Inter', sans-serif;
}}
h1, h2, h3 {{
    color: {uiuc_blue};
    text-align: center;
}}
button {{
    background-color: {uiuc_orange} !important;
    color: white !important;
    border-radius: 10px !important;
    font-weight: 600 !important;
}}
textarea {{
    border: 2px solid {uiuc_blue} !important;
    border-radius: 10px !important;
}}
""") as demo:
    gr.Markdown(f"""
    <div style='text-align:center;'>
        <h1 style='color:{uiuc_blue}; font-size:30px; font-weight:800;'>🎓 UIUC Assistant</h1>
        <p style='color:{uiuc_orange}; font-size:16px; font-weight:500;'>Grounded RAG Chatbot (No Hallucination Mode)</p>
    </div>
    """)

    chatbot = gr.Chatbot(label="Chat with your UIUC Assistant 🤖")

    msg = gr.Textbox(
        label="Ask me about ISSS, Courses, or Campus Life!",
        placeholder="e.g., How can I apply for OPT? or Where is ISR Dining?",
        lines=2
    )

    send = gr.Button("Send")

    def respond(message, chat_history):
        if chat_history is None:
            chat_history = []
        bot_reply = chat_with_bot(message)
        chat_history.append((message, bot_reply))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send.click(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model loaded successfully on A100!


/tmp/ipython-input-2427598678.py:198: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat with your UIUC Assistant 🤖")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c73b5b744461537bf4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
